In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
class person:
    def __init__(self,i,gender, orientation, age, sexual_activity):
        # gender 0 : male, 1 : female
        # orientation 0 : straight, 1 : homosexual
        # time_since_infection equals -1 if not infected
        
        self.identifier = i
        self.gender = gender
        self.orientation = orientation
        self.age = age
        self.sexual_activity = sexual_activity
        self.disease_status = 0
        self.time_since_infection = -1
        self.number_of_partners = 0
        self.current_partners = set()
    
    def print_state(self):
        print(self.identifier, 
              self.gender, 
              self.orientation,
              self.age,
              self.sexual_activity,
              self.time_since_infection,
              self.number_of_partners,
              self.current_partners)
    
    def add_partner(self, other_person):
        if other_person.identifier not in self.current_partners:
            self.number_of_partners += 1
            self.current_partners.add(other_person.identifier)
        
    def remove_partner(self, other_person):
        self.number_of_partners -= 1
        self.current_partners.remove(other_person.identifier)

class Partnership:
    def __init__(self,partnership_type,person1,person2):
        """
        type 1 : steady, 2 : casual
        
        """
        self.type = partnership_type
        self.persons = [person1,person2]
    def print_state(self):
        print(self.type,
              self.persons[0].identifier,
              self.persons[1].identifier)
        
class system:
    def __init__(self):
        self.persons = []
        self.partnerships = []
        self.straight_males = []
        self.homosexual_males = []
        self.females = []
        self.time = 0
    
    def set_constants(self,
                      f = 0.2,
                      rho = 0.006,
                      sigma1 = 0.0004,
                      sigma2 = 0.1,
                      transmission_male_female_steady = 0.15,
                      transmission_female_male_steady = 0.0625,
                      transmission_male_female_casual = 0.6,
                      transmission_female_male_casual = 0.25,
                      ratio_asymptomatic_men = 0.1,
                      ratio_asymptomatic_women = 0.45,
                      incubation_time_men = 5,
                      incubation_time_women = 10,
                      patient_delay_treatment_men = 5,
                      patient_delay_treatment_women = 8,
                      recovery_rate_asymptomatic_men = 0.0074,
                      recovery_rate_symptomatic_men = 0.04,
                      recovery_rate_asymptomatic_women = 0.0044,
                      recovery_rate_symptomatic_women = 0.03,
                      screening_percentage = 0.02):
        """
        sets constants

        Definitions
        f: probability of steady partnership
        rho: probability of forming partnership
        sigma1 : probability of steady partnership separation
        sigma2 : probability of casual partnership separation
        """
        self.f = f
        self.rho = rho
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        self.transmission_male_female_steady = transmission_male_female_steady
        self.transmission_female_male_steady = transmission_female_male_steady
        self.transmission_male_female_casual = transmission_male_female_casual
        self.transmission_female_male_casual = transmission_female_male_casual
        self.ratio_asymptomatic_men = ratio_asymptomatic_men
        self.ratio_asymptomatic_women = ratio_asymptomatic_women
        self.incubation_time_men = incubation_time_men
        self.incubation_time_women = incubation_time_women
        self.patient_delay_treatment_men = patient_delay_treatment_men
        self.patient_delay_treatment_women = patient_delay_treatment_women
        self.recovery_rate_asymptomatic_men = recovery_rate_asymptomatic_men
        self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
        self.recovery_rate_asymptomatic_women = recovery_rate_asymptomatic_women
        self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
        self.screening_percentage = screening_percentage

    
    def initialize(self,number_of_persons, gender_ratio, queer_ratio):
        for i in range(number_of_persons):
            gender =  0 if np.random.random() > gender_ratio else 1
            orientation = 1 if gender == 0 and np.random.random() < queer_ratio else 0
            age = np.random.randint(15,65)
            if age >= 15 and age < 35:
                sexual_activity = 1 if np.random.random() < 0.05 else 0
            else:
                sexual_activity = 0
            
            
            p = person(i,gender, orientation, age, sexual_activity)
            self.persons.append(p)
            if gender == 0 and orientation == 0:
                self.straight_males.append(p)
            elif gender == 0 and orientation == 1:
                self.homosexual_males.append(p)
            else:
                self.females.append(p)
    
    def replace_person(self,p):
        identifier = p.identifier
        gender = p.gender
        orientation = p.orientation
        age = 15
        sexual_activity = 1 if np.random.random() < 0.05 else 0
    
        #remove p from current partners from all p's current partners
        for j in p.current_partners:
            o = self.persons[j]
            o.remove_partner(p)
        
        #remove partnerships where p is involved in
        for i,ps in enumerate(self.partnerships):
            if p.identifier in [ps.persons[0].identifier,ps.persons[1].identifier]:
                self.partnerships[i] = None
        self.partnerships = list(filter(None,self.partnerships))
            
        new_person = person(identifier,gender, orientation, age, sexual_activity)
        
        self.persons[identifier] = new_person
        for i,m in enumerate(self.straight_males):
            if m.identifier == identifier:
                self.straight_males[i] = new_person
        for i,f in enumerate(self.females):
            if f.identifier == identifier:
                self.females[i] = new_person
    
    def print_state(self):
        print("id\tgender\torientation\tage\tactivity\tdisease\ttime since inf\t#partners\tcurrent partners")
        for p in self.persons:
            p.print_state()
        print("type\tpersons")
        for ps in self.partnerships:
            ps.print_state()
            
    def time_step(self):
        n = int(len(self.persons)/2) - len(self.partnerships)
        
        #creation of partnerships
        for i in range(n):
            if np.random.random() < self.rho:
                formed = False
                partnership_type = 1 if np.random.random() < self.f else 2
                
                #add a end condition
                iteration = 0
                while(not formed and iteration < 100):
                    #form partnership
                    y = np.random.choice(self.straight_males)
                    x = np.random.choice(self.females)
                    
                    exists = False
                    mixing_prob = mixing_probability(x,y,partnership_type)
                    if np.random.random() < mixing_prob:
                        for ps in self.partnerships:
                            if y in ps.persons and x in ps.persons:
                                exists = True
                        if not exists:
                            p = Partnership(partnership_type,x,y)
                            x.add_partner(y)
                            y.add_partner(x)
                            self.partnerships.append(p)
                            formed = True
                    iteration += 1
        
        #TODO: Disease transmission
        for i,partnership in enumerate(self.partnerships):
            p1, p2 = partnership.persons
            s1 = p1.disease_status
            s2 = p2.disease_status
            t2 = p2
            partnership_type = partnership.type
            
            if s1 > 0 and s2 == 0:
                infected = p1
                non_infected = p2
            elif s1 == 0 and s2 > 0:
                infected = p2
                non_infected = p1
            else:
                continue
            
            #extend with MSM
                
                
            if infected.gender == 0 and non_infected.gender == 1:
                if partnership_type == 1:
                    if np.random.random() < self.transmission_male_female_steady:
                        non_infected.disease_status = 1
                        non_infected.time_since_infection = 0
                    else:
                        non_infected.disease_status = 2
                        non_infected.time_since_infection = 0
                elif partnership_type == 2:
                    if np.random.random() < self.transmission_male_female_casual:
                        non_infected.disease_status = 1
                        non_infected.time_since_infection = 0
                    else:
                        non_infected.disease_status = 2
                        non_infected.time_since_infection = 0
            elif infected.gender == 1 and non_infected.gender == 0:
                if partnership_type == 1:
                    if np.random.random() < self.transmission_female_male_steady:
                        non_infected.disease_status = 1
                        non_infected.time_since_infection = 0
                    else:
                        non_infected.disease_status = 2
                        non_infected.time_since_infection = 0
                elif partnership_type == 2:
                    if np.random.random() < self.transmission_female_male_casual:
                        non_infected.disease_status = 1
                        non_infected.time_since_infection = 0
                    else:
                        non_infected.disease_status = 2
                        non_infected.time_since_infection = 0

        
        #Separation of partnerships
        for i,ps in enumerate(self.partnerships):
            partnership_type = ps.type
            p1, p2 = ps.persons
            
            #TODO: check type
            if partnership_type == 1:
                if np.random.random() < self.sigma1:
                    p1.remove_partner(p2)
                    p2.remove_partner(p1)
                    self.partnerships[i] = None
            elif partnership_type == 2:
                if np.random.random() < self.sigma2:
                    p1.remove_partner(p2)
                    p2.remove_partner(p1)
                    self.partnerships[i] = None
        self.partnerships = list(filter(None,self.partnerships))
        
        #Replacement
        for p in self.persons:
            if p.age >= 64:
                #replace with new person
                self.replace_person(p)
        
        
        #TODO: Recovery
        for p in self.persons:
            if p.gender == 0 and p.disease_status == 1:
                if np.random.random() < self.recovery_rate_symptomatic_men:
                    p.disease_status = 0
                    p.time_since_infection =  -1
            elif p.gender == 0 and p.disease_status == 2:
                if np.random.random() < self.recovery_rate_asymptomatic_men:
                    p.disease_status = 0
                    p.time_since_infection =  -1
            elif p.gender == 1 and p.disease_status == 1:
                if np.random.random() < self.recovery_rate_symptomatic_women:
                    p.disease_status = 0
                    p.time_since_infection =  -1
            elif p.gender == 1 and p.disease_status == 2:
                if np.random.random() < self.recovery_rate_asymptomatic_women:
                    p.disease_status = 0
                    p.time_since_infection =  -1
                
        #Treatment
        for p in self.persons:
            if p.disease_status == 1:
                if p.gender == 0:
                    if p.time_since_infection >= self.incubation_time_men + self.patient_delay_treatment_men:
                        p.disease_status = 0
                        p.time_since_infection = -1
                        for j in p.current_partners:
                            o = self.persons[j]
                            o.disease_status = 0
                            o.time_since_infection = -1
                elif p.gender == 1:
                    if p.time_since_infection >= self.incubation_time_women + self.patient_delay_treatment_women:
                        p.disease_status = 0
                        p.time_since_infection = -1
                        for j in p.current_partners:
                            o = self.persons[j]
                            o.disease_status = 0
                            o.time_since_infection = -1
        
        
        #Screening
        if self.time % 365 == 0:
            number = int(len(self.persons)*self.screening_percentage)
            screened = np.random.choice(self.persons,number)
            for p in screened:
                p.disease_status = 0
                p.time_since_infection = -1
        
        #Increase time
        self.time += 1
        for p in self.persons:
            if p.disease_status > 0:
                p.time_since_infection += 1
        
        if self.time % 365 == 0:
            for i,p in enumerate(self.persons):
                p.age += 1
        

def mixing_probability(x,y,partnership_type):
    j = age_group(x)
    k = age_group(y)
    
    if j == k:
        phi_a = 1
    else:
        phi_a = 0.2**(np.abs(j + 1 - k))
    
    # c = sexual_activity
    # d = number_of_partners
    
    if partnership_type == 1:
        if x.number_of_partners == 0 and y.number_of_partners == 0:
            phi_cd = 1
        else:
            phi_cd = 0
    else:
        if x.sexual_activity == 1 and y.sexual_activity == 1:
            phi_cd = 1
        elif x.sexual_activity == 1 and y.sexual_activity == 0\
        and y.number_of_partners == 0:
            phi_cd = 0.1
        elif y.sexual_activity == 1 and x.sexual_activity == 0\
        and x.number_of_partners == 0:
            phi_cd = 0.1
        elif x.sexual_activity == 0 and x.number_of_partners == 0\
        and y.sexual_activity == 0 and y.number_of_partners == 0:
            phi_cd = 0.01
        else:
            phi_cd = 0
    return phi_a * phi_cd
        
    
    
def age_group(p):
    a = p.age
    if a >= 15 and a < 25:
        age_group = 1
    elif a >= 25 and a < 35:
        age_group = 2
    elif a >= 35 and a < 45:
        age_group = 3
    elif a >= 45 and a < 55:
        age_group = 4
    else:
        age_group = 5
    return age_group
        
        
        
        
        

            

In [ ]:
s = system()
s.initialize(10,0.5,0)
s.set_constants()
for i in range(10000):
    s.time_step()
s.time = 0


In [ ]:
s.print_state()

In [ ]:
for i in range(1000):
    s.time_step()

In [ ]:
s.print_state()